In [1]:
import pandas as pd
from constants import NOMINAL_COLUMNS, DISCRETE_COLUMNS, CONTINOUS_COLUMNS
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from MLModel import MLModel
import utils

In [2]:
model_handler = MLModel()

fill_values_nominal = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_nominal.pkl')
fill_values_discrete = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_discrete.pkl')
fill_values_continuous = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_continuous.pkl')
min_max_scaler_dict = model_handler.load_model('artifacts/encoders/min_max_scaler_dict.pkl')
onehot_encoders = model_handler.load_model('artifacts/encoders/onehot_encoders_dict.pkl')
xgb = model_handler.load_model('artifacts/models/xgb_model.pkl')

In [3]:
data_path = 'data/horse-colic-train.csv'
df = pd.read_csv(data_path)

In [4]:

sample_data = df.iloc[0]
sample_data = pd.DataFrame([sample_data])

In [5]:
sample_data = sample_data.replace('?', np.nan)
sample_data.columns = ["V" + str(i) for i in range(1, 29)]
sample_data = sample_data.drop(columns=['V3'])

sample_data = utils.create_new_features(sample_data, "V25")
sample_data = utils.create_new_features(sample_data, "V26")
sample_data = utils.create_new_features(sample_data, "V27")

for col in CONTINOUS_COLUMNS:  
    sample_data[col] = pd.to_numeric(sample_data[col], errors='coerce')

for col in NOMINAL_COLUMNS:
    sample_data[col].fillna(fill_values_nominal[col], inplace=True)

for col in DISCRETE_COLUMNS:
    sample_data[col].fillna(fill_values_discrete[col], inplace=True)

for col in CONTINOUS_COLUMNS:
    sample_data[col].fillna(fill_values_continuous[col], inplace=True)

for col, encoder in onehot_encoders.items():
    new_data = encoder.transform(sample_data[col].to_numpy().reshape(-1, 1))
    new_df = pd.DataFrame(new_data, columns=encoder.get_feature_names_out([col]))
    sample_data = pd.concat([sample_data, new_df], axis=1).drop(columns=[col])

for col, scaler in min_max_scaler_dict.items():
    if col in sample_data.columns:
        sample_data[col] = scaler.transform(sample_data[[col]])

if 'V24' in sample_data.columns:
    sample_data = sample_data.drop(columns=['V24'])

predicted_value = xgb.predict(sample_data)

print(f"The predicted value is: {predicted_value[0]}")

The predicted value is: 1


/var/folders/61/cz17lqfd00q9x1d3cz8dd5wh0000gn/T/ipykernel_50216/2206469217.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sample_data[col].fillna(fill_values_nominal[col], inplace=True)
/var/folders/61/cz17lqfd00q9x1d3cz8dd5wh0000gn/T/ipykernel_50216/2206469217.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sample_data[col].fillna(fill_values_nominal[col], inplace=True)
